In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import Select

# Import from Influenster.com

In [4]:
columns = ['phone','author','location','date','grade_5','review']
InfluensterReviews = pd.DataFrame(columns=columns)

In [5]:
def scrapInfluenster(phone_name, phone_id, num_pages) :
    
    columns = ['phone','author','location','date','grade_5','review']
    phone_infos = pd.DataFrame(columns=columns)
    page_infos_phone = []
    page_infos_author = []
    page_infos_location = []
    page_infos_date = []
    page_infos_grade_5 = []
    page_infos_review = []
    
    for i in range(1,num_pages+1):
        
        
        ## Accessing the webpage
        url = "https://www.influenster.com/reviews/"+phone_id+"?review_sort=lowest+grade&review_page="+str(i)
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        ## Descriptions
        descriptions = soup.find_all(class_="content-item-text review-text")
        for i in range(len(descriptions)) :
            descriptions[i] = descriptions[i].get_text()
            
        ## Grades
        stars = soup.find_all(class_="avg-stars")
        grades = []
        for i in (range(len(descriptions))):
            grades += [stars[i+2]["data-stars"]]
            
        ## Dates
        review_date_html = soup.find_all(class_="date")
        dates = []
        for i in (range(len(descriptions))):
            dates += [review_date_html[i]['content']]

        ## Author names
        authors_html = soup.find_all(class_="author-name")
        authors = []
        for i in (range(len(descriptions))):
            authors += [authors_html[i].get_text()]
        
        ## Locations
        authors_location_html = soup.find_all(class_="content-item-author-info")
        locations = []
        for i in range(len(descriptions)) :
            try :
                locations += [authors_location_html[i].find(class_="author-location").get_text()]
            except :
                locations += ['NaN']
        
        columns = ['phone','author','location','date','grade_5','review']
        page_infos = pd.DataFrame(columns=columns)
        
        page_infos_phone += [phone_name for i in range(len(authors))]
        page_infos_author += authors
        page_infos_location += locations
        page_infos_date += dates
        page_infos_grade_5 += grades
        page_infos_review += descriptions
        
        # 1 second pause for not getting caught
        # time.sleep(1)
    
    phone_infos['phone'] = page_infos_phone
    phone_infos['author'] = page_infos_author
    phone_infos['location'] = page_infos_location
    phone_infos['date'] = page_infos_date
    phone_infos['grade_5'] = page_infos_grade_5
    phone_infos['review'] = page_infos_review
        
    return(phone_infos)

In [6]:
toScrap = [\
 ("iPhone X","apple-iphone-x",3),\
 ("iPhone 8 Plus","apple-iphone-8-plus",2),\
 ("iPhone 8","apple-iphone-8",2)\
]
 #("Galaxy S6 edge+ - 64GB - Gold Platinum","samsung-galaxy-s6-edge-64gb-gold-platinum",4),\
 #("Galaxy S6 With 32GB Memory Cell Phone - Black (sprint)","samsung-galaxy-s6-with-32gb-memory-cell-phone-black-sprint",2),\
 #("Galaxy S6 Edge 4g Lte With 32GB Memory Cell Phone - Gold (verizon Wireless)","samsung-galaxy-s6-edge-4g-lte-with-32gb-memory-cell-phone-gold-verizon-wireless",2),\
 #("Galaxy S6 4g With 64GB Memory Cell Phone - White Pearl (at & t)","samsung-galaxy-s6-4g-with-64gb-memory-cell-phone-white-pearl-at-t",2),\
 #("S6 Edge G9250 4G 32GB Unlocked Mobile Phone Green","samsung-galaxy-s6-edge-g9250-4g-32gb-unlocked-mobile-phone-green",2),\
 #("S6 Edge+ 4g Lte With 32GB Memory Cell Phone - Gold Platinum (verizon W...","samsung-galaxy-s6-edge-4g-lte-with-32gb-memory-cell-phone-gold-platinum-verizon-wireless",2),\
 #("S6 Edge With 32GB Memory Cell Phone - Black Sapphire (sprint)","samsung-galaxy-s6-edge-with-32gb-memory-cell-phone-black-sapphire-sprint",1),\
 #("Galaxy S6 4g With 32GB Memory Cell Phone - Black Sapphire (at & t)","samsung-galaxy-s6-4g-with-32gb-memory-cell-phone-black-sapphire-at-t",1),\
 #("Galaxy S6 4g With 32GB Memory Cell Phone - White Pearl (at & t)","samsung-galaxy-s6-4g-with-32gb-memory-cell-phone-white-pearl-at-t",1),\
 #("Galaxy S6 4g Lte With 32GB Memory Cell Phone - White Pearl (verizon Wireless)","samsung-galaxy-s6-4g-lte-with-32gb-memory-cell-phone-white-pearl-verizon-wireless",3),\
 #("Galaxy S6 With 64GB Memory Cell Phone - Black Sapphire (sprint)","samsung-galaxy-s6-4g-lte-with-32gb-memory-cell-phone-white-pearl-verizon-wireless",1),\
 #("Galaxy S6 With 64GB Memory Cell Phone - White Pearl (sprint)","samsung-galaxy-s6-with-64gb-memory-cell-phone-white-pearl-sprint",2),\
 #("Galaxy S6 4g Lte With 32GB Memory Cell Phone - Black Sapphire (verizon Wireless)","samsung-galaxy-s6-4g-lte-with-32gb-memory-cell-phone-black-sapphire-verizon-wireless",1),\
 #("Galaxy S6 Edge 4g With 32GB Memory Cell Phone - Black Sapphire (at & t)","samsung-galaxy-s6-edge-4g-with-32gb-memory-cell-phone-black-sapphire-at-t",2),\
 #("Galaxy S6 Edge 4g With 32GB Memory Cell Phone (unlocked) - White","samsung-galaxy-s6-edge-4g-with-32gb-memory-cell-phone-unlocked-white",1)\
#]

In [7]:
for i in range(len(toScrap)):
    InfluensterReviews = pd.concat([InfluensterReviews, scrapInfluenster(toScrap[i][0], toScrap[i][1], toScrap[i][2])], ignore_index=True)
    print(toScrap[i][0]+' : Done !')

iPhone X : Done !
iPhone 8 Plus : Done !
iPhone 8 : Done !


In [8]:
InfluensterReviews.to_csv('Scrapping_Influenster_16012018.csv', index = False)

In [12]:
InfluensterReviews.head(3)

,phone,author,location,date,grade_5,review
0,iPhone X,Achwek M.,NaN,2018-01-04,3,\n I'm really upset with this e...
1,iPhone X,Damaris V.,"Nayarit, México",2017-12-01,3,\n El precio es muy alto en com...
2,iPhone X,Morgan M.,NaN,2018-01-12,4,\n I’ve had the new iPhone X fo...


# Import from Currys.co.uk

In [142]:
path_to_chromedriver = './chromedriver' # change path as needed
browser = webdriver.Chrome(executable_path = path_to_chromedriver)

In [143]:
url = 'https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-8-64-gb-space-grey-10168742-pdt.html'
browser.get(url)
time.sleep(2)

In [144]:
browser.find_element_by_xpath('//*[@id="product-tabs"]/div[1]/a[3]').click()
time.sleep(2)

In [145]:
select = Select(browser.find_element_by_xpath(\
    '//*[@id="embedded_product_reviews_gtw"]/div[2]/span[2]/span[2]/select'))

In [146]:
select.select_by_visible_text('Lowest score')
time.sleep(2)

In [147]:
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

In [148]:
page_number_html = soup.find(class_="pagination")
max_page = page_number_html.get_text().split()
max_page = int(max_page[len(max_page)-2])
max_page

36

In [149]:
names = []
grades_10 = []
purchase_dates = []
review_dates = []
review_pros = []
review_cons = []

for i in range(max_page-1):
    
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    names_html = soup.find_all(class_="attribution-name")
    for name in names_html :
        names += [name.get_text()]
    
    grades_10_html = soup.find_all(class_="overall_score_stars")
    for grade in grades_10_html :
        grades_10 += [grade['title']]
    
    purchase_date_html = soup.find_all(class_="purchase_date")
    for date in purchase_date_html :
        date = date.get_text().split()
        purchase_dates += [date[2]+' '+date[3]+' '+date[4]]
        review_dates += [date[7]+' '+date[8]+' '+date[9]]
    
    review_pros_html = soup.find_all(class_="pros")
    i=0
    for pros in review_pros_html :
        i+=1
        if i%2 ==0 :
            review_pros += [pros.get_text()]
            
    review_cons_html = soup.find_all(class_="cons")
    i=0
    for cons in review_cons_html :
        i+=1
        if i%2 ==0 :
            review_cons += [cons.get_text()]
    
    browser.find_element_by_class_name('next_page').click()
    time.sleep(3)

columns = ['author','grade_10','purchase_date','review_date','review_pros','review_cons']
phone_data = pd.DataFrame(columns=columns)

phone_data.author = names
phone_data.grade_10 = grades_10
phone_data.purchase_date = purchase_dates
phone_data.review_date = review_dates
phone_data.review_pros = review_pros
phone_data.review_cons = review_cons

phone_data

,author,grade_10,purchase_date,review_date,review_pros,review_cons
0,Paul,3,20 October 2017,20 November 2017,Easy to use and reliable,Camera disappointing bad in low light. Phone c...
1,Shafiq,3,08 October 2017,20 November 2017,It’s easy to use like all iPhones,"Too delicate Too many software issues , mobile..."
2,Benjamin,3,11 October 2017,12 November 2017,"It’s exactly the same as my previous iphone, w...",The phone is alot louder tjen many others but ...
3,Muhammad,3,20 September 2017,05 October 2017,Nothing. Rip-off,Nothing new in the phone except the glass .
4,Fabio,4,24 October 2017,21 November 2017,Very little change from previous iPhone. After...,Very little change from previous iPhone. After...
5,Glen,4,19 October 2017,20 November 2017,None,It’s exactly the same as the 6s that I already...
6,Philip,4,20 October 2017,20 November 2017,"Great camera, 3D Touch and Apple Pay all work ...",Losing contacts purely because the latest upda...
7,Viktorija,5,27 October 2017,28 November 2017,It's new. Batter lasts a long time. Storange s...,Nothing that would make it worth the money!! S...
8,Confirmed purchaser,5,18 October 2017,20 November 2017,same layout as 6s,not changed a lot since earlier models
9,Ryan,5,16 October 2017,17 November 2017,Apple brand,Upgrading from 5s to 8 is not worth the extra ...


In [ ]:
phone_data.to_csv('Scrapping_currys_16012018.csv', index = False)

In [184]:
def scrapCurrys(phone, url):
    
    
    phones = []
    names = []
    grades_10 = []
    purchase_dates = []
    review_dates = []
    review_pros = []
    review_cons = []
    
    
    browser.get(url)
    time.sleep(2)
    
    browser.find_element_by_xpath('//*[@id="product-tabs"]/div[1]/a[3]').click()
    time.sleep(2)
    
    select = Select(browser.find_element_by_xpath(\
        '//*[@id="embedded_product_reviews_gtw"]/div[2]/span[2]/span[2]/select'))
    select.select_by_visible_text('Lowest score')
    time.sleep(2)
    
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    page_number_html = soup.find(class_="pagination")
    max_page = page_number_html.get_text().split()
    max_page = int(max_page[len(max_page)-2])
    
    length = 0
    
    for k in range(max_page-1):
    
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')

        names_html = soup.find_all(class_="attribution-name")
        for name in names_html :
            names += [name.get_text()]

        grades_10_html = soup.find_all(class_="overall_score_stars")
        for grade in grades_10_html :
            grades_10 += [grade['title']]

        purchase_date_html = soup.find_all(class_="purchase_date")
        for date in purchase_date_html :
            date = date.get_text().split()
            purchase_dates += [date[2]+' '+date[3]+' '+date[4]]
            review_dates += [date[7]+' '+date[8]+' '+date[9]]

        review_pros_html = soup.find_all(class_="pros")
        i=0
        for pros in review_pros_html :
            i+=1
            if i%2 ==0 :
                review_pros += [pros.get_text()]

        review_cons_html = soup.find_all(class_="cons")
        i=0
        for cons in review_cons_html :
            i+=1
            if i%2 ==0 :
                review_cons += [cons.get_text()]

        browser.find_element_by_class_name('next_page').click()
        time.sleep(3)
        
        length += len(names_html)

    columns = ['phone','author','grade_10','purchase_date','review_date','review_pros','review_cons']
    phone_data = pd.DataFrame(columns=columns)
    
    phone_data.phone = [phone for n in range(length)]
    phone_data.author = names
    phone_data.grade_10 = grades_10
    phone_data.purchase_date = purchase_dates
    phone_data.review_date = review_dates
    phone_data.review_pros = review_pros
    phone_data.review_cons = review_cons

    return(phone_data)

In [185]:
toScrap = [\
 ("iPhone X","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-x-64-gb-space-grey-10168683-pdt.html"),\
 ("iPhone X","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-x-64-gb-silver-10168684-pdt.html"),\
 ("iPhone X","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-x-256-gb-space-grey-10168685-pdt.html"),\
 ("iPhone X","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-x-256-gb-silver-10168686-pdt.html"),\
 ("iPhone 8","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-8-64-gb-space-grey-10168742-pdt.html"),\
 ("iPhone 8","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-8-64-gb-gold-10168743-pdt.html"),\
 ("iPhone 8+","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-8-plus-64-gb-space-grey-10168678-pdt.html"),\
 ("iPhone 8+","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-8-plus-64-gb-silver-10168677-pdt.html"),\
 ("iPhone 8+","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-8-plus-64-gb-gold-10168679-pdt.html"),\
 ("iPhone 8","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-8-256-gb-space-grey-10168675-pdt.html"),\
 ("iPhone 8","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-8-64-gb-silver-10168741-pdt.html"),\
 ("iPhone 8+","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-8-plus-256-gb-gold-10168682-pdt.html"),\
 ("iPhone 8+","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-8-plus-256-gb-space-grey-10168681-pdt.html"),\
 ("iPhone 8+","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-8-plus-256-gb-silver-10168680-pdt.html"),\
 ("iPhone 8","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-8-256-gb-gold-10168676-pdt.html"),\
 ("iPhone 8","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/apple-iphone-8-256-gb-silver-10168744-pdt.html"),\
 ("Galaxy S8","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/samsung-galaxy-s8-64-gb-coral-blue-10168236-pdt.html"),\
 ("Galaxy S8+","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/samsung-galaxy-s8-64-gb-black-10161804-pdt.html"),\
 ("Galaxy S8","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/samsung-galaxy-s8-64-gb-black-10161802-pdt.html"),\
 ("Galaxy S8","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/samsung-galaxy-s8-64-gb-pink-gold-10168727-pdt.html"),\
 ("Galaxy S8+","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/samsung-galaxy-s8-64-gb-coral-blue-10168237-pdt.html"),\
 ("Galaxy S8+","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/samsung-galaxy-s8-64-gb-orchid-grey-10161803-pdt.html"),\
 ("Galaxy S8+","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/samsung-galaxy-s8-64-gb-pink-gold-10168726-pdt.html"),\
 ("Galaxy S8","https://www.currys.co.uk/gbuk/phones-broadband-and-sat-nav/mobile-phones-and-accessories/mobile-phones/samsung-galaxy-s8-64-gb-orchid-grey-10161801-pdt.html")\
]

In [186]:
columns = ['phone','author','grade_10','purchase_date','review_date','review_pros','review_cons']
currys_data = pd.DataFrame(columns=columns)

In [192]:
path_to_chromedriver = './chromedriver' # change path as needed
browser = webdriver.Chrome(executable_path = path_to_chromedriver)

In [193]:
for scrap in toScrap:
    currys_data = pd.concat([currys_data, scrapCurrys(scrap[0], scrap[1])], ignore_index=True)

currys_data.to_csv('Scrapping_currys_16_01_2018.csv', index = False)

In [201]:
currys_data=currys_data[currys_data.review_cons!="Reviewer left no comment"]
currys_data.to_csv('Scrapping_currys_16_01_2018.csv', index = False)

In [202]:
currys_data=pd.read_csv('Scrapping_currys_16_01_2018.csv',sep=',')

In [203]:
currys_data

,phone,author,grade_10,purchase_date,review_date,review_pros,review_cons
0,iPhone X,Sara,5,13 November 2017,23 November 2017,"Quick, and easy to use. Battery life is decent...",The screen is very easy to break. Broke my scr...
1,iPhone X,Dorji,8,14 November 2017,24 November 2017,"I love special features like face recognition,...",It would have been awesome if it has finger sc...
2,iPhone X,Shelly,8,13 November 2017,24 November 2017,early to say yet but generally better,face recognition doesnt work properly my son a...
3,iPhone X,Jijo,9,14 November 2017,28 November 2017,The screen looks amazing. The colors are vibra...,"Honestly, I do not have a genuine complaint at..."
4,iPhone X,Mark,9,14 November 2017,24 November 2017,"Fantastic portrait camera mode, its hard to ta...",Its a personal thing but the screen is too lar...
5,iPhone X,Mihai,9,06 November 2017,20 November 2017,Good phone Looks amazingFeels great in hand Lo...,Have to retrain muscle memory : easy after a c...
6,iPhone X,Paul,10,15 December 2017,15 January 2018,Best iPhone ever and best screen I’ve seen on ...,Price
7,iPhone X,Graeme,10,24 November 2017,12 January 2018,"feels nice to use, battery life and screen res...",takes a little while to get used to lack of ho...
8,iPhone X,Kim,10,12 December 2017,12 January 2018,Fab Camera my pictures are framed in a 10x 8 ...,The price .
9,iPhone X,Marcin,10,23 November 2017,12 January 2018,"Very fast,beautiful photos and new options",No ear pods
